Wadi Moughanim - Lecture 1 (2023/10/05): Exercise 1.3

### Exercise 1.3 (Domains of Attraction: Illustration)


### 1. Illustrate the Phenomenon of Weak Convergence
Illustrate the phenomenon of weak convergence stated in Fisher and Tippett’s theorem through the convergence of histograms (built from a random sample) of maxima towards histograms of the limit, for a negative shape parameter.

- **Choose a Distribution:** Choose a textbook distribution F in the Weibull domain of attraction and find appropriate norming sequences an, bn such that (MDA) holds.

We aim to show that the uniform distribution on the interval (0,1) belongs to the Weibull domain of attraction. We consider the cumulative distribution function (CDF) of $U(0,1)$,

$$
F_{\text{unif}}(x) = x \cdot \mathbb{1}_{(0,1)}(x),
$$

We proceed by choosing sequences $a_n$ and $b_n$ such that 

$$
a_n x + b_n \to \infty \quad \text{as} \quad n \to \infty.
$$

Then : 

$$
F^n(a_n x + b_n) = (a_n x + b_n)^n,
$$

valid for all $x \leq \frac{1 - b_n}{a_n}$

Let’s choose $b_n = 1$ and allow $a_n$ to be any non-zero sequence and $x<0$:

$$
F^n(a_n x + b_n) = \exp\left(n \log(1 + a_n x)\right).
$$

For a specific choice of $a_n = \frac{1}{n}$, the expression simplifies to

$$
F^n\left(\frac{x}{n} + 1\right) \approx \exp(x) \mathbb{1}_{x<0}
$$

This is a particular case of the Weibull distribution with $\alpha = -1$. Hence, with the sequences $a_n = \frac{1}{n}$ and $b_n = 1$.

---
- **Write a Short Code:** Write a short code allowing to:
    - Generate M blocks of size n of independent random variables distributed according to F and normalize the block maxima.
    - Plot a histogram of the M normalized maxima and superimpose the histogram for the limit distribution in a visually illustrative manner.


In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n = 30  # Size of each block

# Generate M blocks each of size n from U(0,1)
def generate_blocks(M, n):
    return np.random.rand(M, n)

# Compute block maxima
def compute_maxima(blocks):
    return np.max(blocks, axis=1)

# Normalize the block maxima
def normalize_maxima(maxima, n):
    a_n = 1.0 / n
    b_n = 1
    return (maxima - b_n)/a_n


def plot_analysis(blocks, block_maxima, normalized_maxima, M, nbins):
    block_ends = np.arange(n, M*n+1, n)
    raw_data = blocks.flatten()
    fig = make_subplots(rows=3, cols=1, subplot_titles=(f"Block Maxima and Raw Data with {M} Observations", f"Normalized Maxima with {M} Observations", f"Histogram of Normalized Maxima with Limit Distribution for {M} Observations"))
    fig.add_trace(
        go.Scatter(y=raw_data, mode='markers', marker=dict(color='black', size=6), name='Raw data', showlegend=False),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=block_ends-n/2, y=block_maxima, mode='markers', marker=dict(symbol='cross', color='red', size=10), name='Block Maxima'),
        row=1, col=1
    )
    for end in block_ends:
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=end,
                x1=end,
                y0=0,
                y1=1.2,  
                line=dict(color="blue"),
            ),
            row=1, col=1
        )
    fig.add_trace(
        go.Scatter(y=normalized_maxima, mode='lines+markers', name=f"Normalized Maxima with {M} Observations"),
        row=2, col=1
    )
    fig.add_trace(
        go.Histogram(x=normalized_maxima, name=f"Histogram of Normalized Maxima with {M} Observations", nbinsx=nbins, histnorm='probability density'),
        row=3, col=1
    )
    # Exponential trace (limit distribution)
    x_vals = np.linspace(min(normalized_maxima), max(normalized_maxima), 400)
    y_vals = np.exp(x_vals)  
    fig.add_trace(
        go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Limit Distribution'),
        row=3, col=1
    )

    fig.update_layout(title_text=f"Analysis for {M} Observations", height=900)
    fig.show()

# 1
M1 = 100
blocks1 = generate_blocks(M1, n)
block_maxima1 = compute_maxima(blocks1)
normalized_maxima1 = normalize_maxima(block_maxima1, n)
plot_analysis(blocks1, block_maxima1, normalized_maxima1, M1, nbins=100)

# 2
M2 = 300
blocks2 = generate_blocks(M2, n)
block_maxima2 = compute_maxima(blocks2)
normalized_maxima2 = normalize_maxima(block_maxima2, n)
plot_analysis(blocks2, block_maxima2, normalized_maxima2, M2, nbins=100)


- **Vary M and n:** Let M and n vary so as to illustrate weak convergence of maxima as M → ∞. Explain the role of M and n in what you observe. Summarize the results in a figure including ≈ 6 such histograms with different values of M and a single (appropriate) value of n.

In [2]:
n_values = [10, 20, 40, 80, 160, 320]
M_values = [50, 100, 200, 400, 800, 1600]

# Setup for the summary figure
fig = make_subplots(rows=6, cols=1, subplot_titles=[f"Histogram for M = {M}" for M in M_values])

# Populate the summary figure
for idx, (M, n) in enumerate(zip(M_values, n_values)):
    blocks = generate_blocks(M, n)
    block_maxima = compute_maxima(blocks)
    normalized_maxima = normalize_maxima(block_maxima, n)
    x_vals = np.linspace(min(normalized_maxima), max(normalized_maxima), 400)
    y_vals = np.exp(x_vals)
    
    fig.add_trace(go.Histogram(x=normalized_maxima, name=f'M = {M}', nbinsx=100, histnorm='probability density'), row=idx+1, col=1)
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Limit Distribution', showlegend=False), row=idx+1, col=1)

fig.update_layout(height=1800, title_text="Histograms for Different M values", bargap=0.1)
fig.show()

### 2. Uniform Convergence of c.d.f’s
Show (graphically and numerically) uniform convergence of c.d.f.’s. Explain why (i.e., prove that) weak convergence of normalized maxima indeed implies uniform convergence of c.d.f’s.

The cdf. $F_n$ of the normalized maxima converges to $F$ at all cadlag points of $F$. Given that $F$ is cadlag everywhere in the 3 cases Fréchet, Weibull and Gumbel, $F_n$ converges to $F$ for every $x \in \mathbb{R}$.

As F is increasing and for any $\epsilon > 0$, we take points $-\infty = x_0 < x_1 < \cdots < x_k = +\infty$ such that $\forall i \in =0,...,k-1$ $F(i+1)-F(i) \leq \epsilon$ 

For any sub-interal $[x_i,x_{i+1}]$ of $[x_0,x_k]$, we have $F(x_{i+1})-F(x_i) \leq \epsilon$.
Thus as both $F_n$ and $F$ are increasing and bounded, we have $sup_{x  \in [x_i,x_{i+1}]} |F_n(x) - F(x)| \leq max(|F_n(x_i) - F(x_i)|,|F_n(x_{i+1}) - F(x_{i+1})|) + \epsilon$.
$$
Thus:
\sup_{x \in \mathbb{R}} |F_n(x) - F(x)| \leq \max_{j=0,1,\dots,K} |F_n(x_j) - F(x_j)| + \epsilon
$$
 As $n$  -> $\infty$, the term $\max_{j=0,1,\dots,K} |F_n(x_j) - F(x_j)|$ goes to 0 due to weak convergence. Thus, for sufficiently large $n$, the supremum of the absolute differences can be made smaller than any given $\epsilon$, implying uniform convergence.


In [9]:
# Parameters
n = 30  # Size of each block

def generate_blocks(M, n):
    return np.random.rand(M, n)

def compute_maxima(blocks):
    return np.max(blocks, axis=1)

def normalize_maxima(maxima, n):
    a_n = 1.0 / n
    b_n = 1
    return (maxima - b_n) / a_n

def empirical_cdf(data):
    sorted_data = np.sort(data)
    y = np.arange(1, len(data) + 1) / len(data)
    return sorted_data, y

def weibull_cdf(x):
    return np.exp(x)

def plot_convergence(M_values):
    fig = make_subplots(rows=len(M_values), cols=1)

    for i, M in enumerate(M_values):
        blocks = generate_blocks(M, n)
        maxima = compute_maxima(blocks)
        normalized_maxima = normalize_maxima(maxima, n)
        
        x_empirical, y_empirical = empirical_cdf(normalized_maxima)
        
        x_vals = np.linspace(min(normalized_maxima), max(normalized_maxima), 400)
        y_vals = weibull_cdf(x_vals)
        
        fig.add_trace(go.Scatter(x=x_empirical, y=y_empirical, mode='lines', name=f'Empirical CDF M={M}'), row=i+1, col=1)
        fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Gumbel CDF'), row=i+1, col=1)
        
    fig.update_layout(title_text="Uniform Convergence of CDFs", height=200*len(M_values))
    fig.show()

M_values = [30, 100, 300, 1000, 3000, 10000]
plot_convergence(M_values)


### 3. Translated Pareto Distribution
Change the input distribution and work with a translated Pareto distribution, $ P(X > x) = \left(\frac{x - \beta}{u}\right)^{- \alpha} $, for some $ \alpha > 0, \beta, u \in \mathbb{R}, x \geq u + \beta $. Draw similar outputs as in the previous questions and compare the rate of convergence.

$$
F(x) = 1 - \left( \frac{x - \beta}{u} \right)^{-\alpha} \quad \text{for} \quad x > u + \beta,
$$

we are looking to find sequences $a_n$ and $b_n$ that normalize maxima.

$$
P(X > x) = \left( \frac{x - \beta}{u} \right)^{-\alpha}.
$$

Given the normalization $F_n(a_n x + b_n)$, and setting $b_n = \beta$:

$$
F_n(a_n x + \beta) = \left(1 - \left( \frac{a_n x + \beta - \beta}{u} \right)^{-\alpha}\right)^n
$$

$$
F_n(a_n x + \beta) = \left(1 - \left( \frac{a_n x}{u} \right)^{-\alpha}\right)^n.
$$

For large $n$, this behaves as:

$$
F_n(a_n x + \beta) \approx \exp\left(n \log\left(1 - \left( \frac{a_n x}{u} \right)^{-\alpha}\right)\right).
$$

For the normalization to converge to the Fréchet distribution, we aim for a structure that's proportional to $\exp(-x^{-\alpha})$.

To attain this, let's choose $a_n = u n^{1/\alpha}$. Plugging in this value:

$$
F_n(u n^{1/\alpha} x + \beta) \approx \exp\left(n \log\left(1 - \left( \frac{n^{1/\alpha} x}{u} \right)^{-\alpha}\right)\right).
$$

For large $n$, the term inside the logarithm gives us the desired structure, $\exp(-x^{-\alpha})$, which is the Fréchet distribution.

In [4]:
n = 30  # Size of each block

# List of parameter sets
parameter_sets = [
    {"alpha": 2.0, "beta": 0.5, "u": 1.0},
    {"alpha": 2.5, "beta": 0.7, "u": 1.2},
    {"alpha": 3.0, "beta": 0.9, "u": 1.5}
]

def generate_blocks(M, n, alpha, beta, u):
    # Generate uniform random numbers
    uniform_samples = np.random.rand(M, n)
    # Use the inverse CDF to get Pareto samples
    pareto_samples = beta + u * ((1 - uniform_samples)**(-1/alpha))
    return pareto_samples

def compute_maxima(blocks):
    return np.max(blocks, axis=1)

def normalize_maxima(maxima, n):
    a_n = u * n**(1/alpha)
    b_n = beta
    return (maxima - b_n) / a_n

def plot_analysis(blocks, block_maxima, normalized_maxima, M, nbins, alpha, beta, u):
    block_ends = np.arange(n, M*n+1, n)
    raw_data = blocks.flatten()
    fig = make_subplots(rows=3, cols=1, subplot_titles=(f"Block Maxima and Raw Data with {M} Observations", 
                                                        f"Normalized Maxima with {M} Observations", 
                                                        f"Histogram of Normalized Maxima with Limit Distribution for {M} Observations"))

    fig.add_trace(
        go.Scatter(y=raw_data, mode='markers', marker=dict(color='black', size=6), name='Raw data', showlegend=False),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=block_ends-n/2, y=block_maxima, mode='markers', marker=dict(symbol='cross', color='red', size=10), name='Block Maxima'),
        row=1, col=1
    )

    for end in block_ends:
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=end,
                x1=end,
                y0=0,
                y1=1.2,  
                line=dict(color="blue"),
            ),
            row=1, col=1
        )
    fig.add_trace(
        go.Scatter(y=normalized_maxima, mode='lines+markers', name=f"Normalized Maxima with {M} Observations"),
        row=2, col=1
    )

    fig.add_trace(
        go.Histogram(x=normalized_maxima, name=f"Histogram of Normalized Maxima with {M} Observations", nbinsx=nbins, histnorm='probability density'),
        row=3, col=1
    )
    
    # Frechet PDF (theoretical)
    x_vals = np.linspace(min(normalized_maxima), max(normalized_maxima), 400)
    y_vals = alpha * x_vals**(-alpha - 1) * np.exp(-x_vals**(-alpha))
    fig.add_trace(
        go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Theoretical Frechet PDF'),
        row=3, col=1
    )

    title = f"Analysis for {M} Observations (α={alpha}, β={beta}, u={u})"
    fig.update_layout(title_text=title, height=900)
    fig.show()


for params in parameter_sets:
    alpha, beta, u = params["alpha"], params["beta"], params["u"]
    blocks = generate_blocks(M1, n, alpha, beta, u)
    block_maxima = compute_maxima(blocks)
    normalized_maxima = normalize_maxima(block_maxima, n)
    plot_analysis(blocks, block_maxima, normalized_maxima, M1, nbins=100, alpha=alpha, beta=beta, u=u)

In [5]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Parameters
n = 30  # Size of each block

# List of parameter sets
parameter_sets = [
    {"alpha": 2.0, "beta": 0.5, "u": 1.0},
    {"alpha": 2.5, "beta": 0.7, "u": 1.2},
    {"alpha": 3.0, "beta": 0.9, "u": 1.5}
]

def generate_blocks(M, n, alpha, beta, u):
    # Generate uniform random numbers
    uniform_samples = np.random.rand(M, n)
    # Use the inverse CDF to get Pareto samples
    pareto_samples = beta + u * ((1 - uniform_samples)**(-1/alpha))
    return pareto_samples

def compute_maxima(blocks):
    return np.max(blocks, axis=1)

def normalize_maxima(maxima, n, alpha, u, beta):
    a_n = u * n**(1/alpha)
    b_n = beta
    return (maxima - b_n) / a_n

def empirical_cdf(data):
    sorted_data = np.sort(data)
    y = np.arange(1, len(data) + 1) / len(data)
    return sorted_data, y

def frechet_cdf(x, alpha):
    return np.exp(-x**(-alpha))

def plot_convergence(M_values, alpha, beta, u):
    fig = make_subplots(rows=len(M_values), cols=1, subplot_titles=[f"M = {M}" for M in M_values])

    for i, M in enumerate(M_values):
        blocks = generate_blocks(M, n, alpha, beta, u)
        maxima = compute_maxima(blocks)
        normalized_maxima = normalize_maxima(maxima, n, alpha, u, beta)
        
        x_empirical, y_empirical = empirical_cdf(normalized_maxima)
        
        x_vals = np.linspace(min(normalized_maxima), max(normalized_maxima), 400)
        y_vals = frechet_cdf(x_vals, alpha)
        
        fig.add_trace(go.Scatter(x=x_empirical, y=y_empirical, mode='lines', name=f'Empirical CDF M={M}'), row=i+1, col=1)
        
        # Add the theoretical Frechet CDF to the subplot
        fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Theoretical Fréchet CDF', line=dict(dash='dash')), row=i+1, col=1)

    # Update layout and show the figure
    fig.update_layout(title_text=f"Uniform Convergence of CDFs with α={alpha}, β={beta}, u={u}", height=200*len(M_values))
    fig.show()

# List of sample sizes to analyze
M_values = [3, 10, 30, 100, 300, 1000]

# Iterate over parameter sets and plot the convergence for each
for params in parameter_sets:
    alpha, beta, u = params["alpha"], params["beta"], params["u"]
    plot_convergence(M_values, alpha, beta, u)


Now let's see the differnece between the rates of convergence

In [6]:
def ks_statistic(empirical_x, empirical_y, theoretical_cdf, alpha):
    """Compute the Kolmogorov-Smirnov statistic."""
    theoretical_y = [theoretical_cdf(x, alpha) for x in empirical_x]
    return np.max(np.abs(empirical_y - theoretical_y))

def plot_ks_statistics(M_values, parameter_sets):
    """Plot the KS statistics for different sample sizes and parameter sets."""
    fig = go.Figure()
    
    for params in parameter_sets:
        alpha, beta, u = params["alpha"], params["beta"], params["u"]
        ks_stats = []
        
        for M in M_values:
            blocks = generate_blocks(M, n, alpha, beta, u)
            maxima = compute_maxima(blocks)
            normalized_maxima = normalize_maxima(maxima, n, alpha, u, beta)
            x_empirical, y_empirical = empirical_cdf(normalized_maxima)
            
            ks = ks_statistic(x_empirical, y_empirical, frechet_cdf, alpha)
            ks_stats.append(ks)
        
        fig.add_trace(go.Scatter(x=M_values, y=ks_stats, mode='lines+markers', name=f'α={alpha}, β={beta}, u={u}'))

    fig.update_layout(title='Rate of Convergence', xaxis_title='Sample Size', yaxis_title='KS Statistic', xaxis_type="log")
    fig.show()

plot_ks_statistics(M_values, parameter_sets)


### 4. Estimate the GEV Parameters
With the distribution from question 1 or 3, generate a dataset of an appropriate size and estimate the GEV parameters with a maximum-likelihood method. Discuss the convergence towards the true parameters.


Let's pick up the pareto distribution from question 3 and generate a dataset of size 10000.


In [7]:
M = 10000  
alpha_true = 2.0
beta_true = 0.5
u_true = 1.0

blocks = generate_blocks(M, n, alpha_true, beta_true, u_true)
maxima = compute_maxima(blocks)

from scipy.stats import genextreme as gev
from scipy.optimize import minimize

# Negative log-likelihood for the GEV distribution
def negative_gev_log_likelihood(params, data):
    c, loc, scale = params
    return -np.sum(gev.logpdf(data, c, loc=loc, scale=scale))

# Maximum-likelihood estimation
initial_guess = [0.5, np.mean(maxima), np.std(maxima)]
res = minimize(negative_gev_log_likelihood, initial_guess, args=(maxima,))
c_hat, loc_hat, scale_hat = res.x

print(f"Estimated parameters: c = {c_hat:.4f}, loc = {loc_hat:.4f}, scale = {scale_hat:.4f}")


Estimated parameters: c = 0.5000, loc = 9.9489, scale = 11.3879


/Users/wadimoughanim/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning:

invalid value encountered in subtract



We obtain a shape value of 0.5000, a location (which is a bit like an average) of 10.3723 and a scale (which indicates how scattered our data are) of 15.1434.